Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) python packages needed: compress_pickle[lz4] pandas numpy yahooquery seaborn

2) From the stock dataset, copy "Stock_List.csv" and "Stocks/*" to "data/". You can delete the other files as they are not used (at least yet).

In [1]:
try:
    import pandas as pd
    import numpy as np
    import seaborn
    from glob import glob
    from compress_pickle import dump, load
    import os
    from yahooquery import Ticker
    import timeit
    import time
    import gc, os
    import datetime
    from getpass import getpass
except ModuleNotFoundError as e:
    print(e)
    print('Please install the missing module(s)')
    print("pip install compress_pickle[lz4] pandas numpy yahooquery cryptography seaborn")
    
from utils import fernet_key_encryption, aquire_stock_search_terms as aquire_terms, get_macroeconomic_data as macro_data

In [ ]:
# This section will first ask you to set a password. After this, you will be prompted to enter any api keys to store in an encrypted file. 
# After this first run, you will only need to enter the password to load the api keys.
# If you need to change the keys or password, delete the relevent .secret keys file and run this section again.
# salt.secret is a non-sensitive file that is used to both generate the encryption key as well as decryption. If this key is lost, the encrypted files are lost and you will need to re-enter the api keys.


# Ask for input of password to save API keys
password = getpass("Enter password to save/load API keys: ");

# Ask for Kaggle API key
kaggle_api_key = fernet_key_encryption(password, 'Kaggle')

# Ask for TD Ameritrade API key
#td_ameritrade_api_key = fernet_key_encryption(password, 'TD Ameritrade')

data_nasdaq = fernet_key_encryption(password, 'Nasdaq')



del password
gc.collect()

In [ ]:
# Gather the company info for all the ticker symbols and return a dataframe with relevant search terms for each company.
search_terms = aquire_terms('data/Stocks/')
search_terms.data

In [ ]:
# gether historical macroeconomic data from different sources.
macro_data = macro_data('data/Macro/')

Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy